## Basic Case

In [10]:
# 1. Load configuration file
import json
with open('../data_files/config_1hrad.json') as f:
     config = json.load(f)
    
import sys
sys.path.append('../')

# 2. Import SIMOC
from agent_model import AgentModel

# 3. Initialize model
model = AgentModel.from_config(config, data_collection=True)

# 4. Step the model forward
model.step_to(n_steps=601)

# 5. Get data from model
data = model.get_data(debug=True)

# 6. Inspect data
import matplotlib.pyplot as plt
radish_growth = data['radish']['growth']['growth_rate']
plt.plot([i for i in range(len(radish_growth))], radish_growth)

ImportError: cannot import name 'AgentModel' from 'agent_model' (unknown location)

## Custom Agent

Define an example agent, mushrooms.
- Consumes 10g o2 and 10g inedible_biomass per hour (sigmoid)
- Grow 20g per hour (normal)
- Lifetime of 30 days
- At end of lifetime, 90% edible (mushroom) 10% waste (mushroom_SMS)

In [1]:
# I specify 2 new currencies:
currency_desc = {
    'food': {
        # The edible product of the mushroom plant
        'algae': {
            'label': 'Algae'
            # Here I could also specify nutrition data
        }
    },
#    'nutrient_storage': {
#        # The unconsumed portion of the algal biomass.
#        'algae_waste': {
#            'label': 'Waste Algae Biomass'
#        }
#    },
#        'nutrients': {
#        # The unconsumed portion of the algal biomass.
#        'algae_waste': {
#            'label': 'Waste Algae Biomass'
#        }
#    },
#    'nutrients': {
#        # The unconsumed portion of the algal biomass.
#        'nutrient_sol': {
#            'label': 'Algae Nutrient Solution'
#        }
#    }
}

agent_desc = {
    # Add a capacity to store these currencies in the appropriate agent
    'storage': {
        'food_storage': {
            'data': {
                'characteristics': [
                    {
                        'type': 'capacity_algae',
                        'value': 1000,
                        'unit': 'kg'
                    }
                ]
            }
        },
#        'nutrient_storage': {
#            'data': {
#                'characteristics': [
#                    {
#                        'type': 'capacity_algae_waste',
#                        'value': 1000,
#                        'unit': 'kg'
#                    }
#                ]
#            }
#        }
    },
    # Agents under 'plants' have lifecycle, growth & reproduction
    'plants': {
        'algae': {
            'data': {
                'input': [
                    {
                        # Matches a currency
                        'type': 'co2',
                        # Average value over life of plant
                        'value': 0.0000059,
                        # If amount in storage is less than demand at any step:
                        # - if required == 'desired', increment deprive (see below),
                        #   don't increment age, but process other currencies as normal
                        # - if required == 'mandatory', exit step function with no side-effects
                        # - if not required, ignore and continue as normal
                        'required': 'desired',
                        'flow_rate': {
                            'unit': 'kg',
                            'time': 'hour'
                        },
                        'growth': {
                            'lifetime': {
                                'type': 'sigmoid',
                                'steepness': 0.2    # Value is 'reshaped' over lifetime
                            }
                        },
                        # If demand is unfulfilled for longer than this, agent dies.
                        'deprive':  {
                            'value': 72,
                            'unit': 'hour'
                        }
                    },
#                    {
#                        'type': 'nutrient_sol',
#                        'value': 0.01,
#                        'required': 'desired',
#                        'flow_rate': {
#                            'unit': 'kg',
#                            'time': 'hour',
#                        },
#                        'growth': {
#                            'lifetime': {
#                                'type': 'sigmoid'
#                            }
#                        },
#                        'deprive': {
#                            'value': 72,
#                            'unit': 'hour'
#                        }
#                    },
                    {
                        # This field and output:mushroom/mushroom_SMS comprise 
                        # 'harvest': when grown == True (aka age == lifetime), consume 
                        # internal biomass and convert to food or waste (as outputs).
                        # Current_growth is kg of accumulated biomass, and 'weighted'
                        # multiplies that value by the input value (in this case 1).
                        'type': 'biomass',
                        'value': 1,
                        'weighted': 'current_growth',
                        'flow_rate': {
                            'unit': 'kg',
                            'time': 'hour',
                        },
                        'criteria': {
                            'name': 'grown',
                            'limit': '=',
                            'value': True
                        }
                    }
                ],
                'output': [
                    {
                        # Plants 'store' their accumulated biomass internally. See 
                        # the 'capacity_biomass' characteristic, and the connection
                        # {from: self.biomass, to: self.biomass}.
                        'type': 'biomass',
                        'value': 0.02,
                        'flow_rate': {
                            'unit': 'kg',
                            'time': 'hour'
                        },
                        'growth': {
                            'lifetime': {
                                'type': 'norm'
                            }
                        }
                    },
                    {                        
                        'type': 'o2',
                        'value': 0.0000065,
                        'flow_rate': {
                            'unit': 'kg',
                            'time': 'hour'
                        },
                        'growth': {
                            'lifetime': {
                                'type': 'sigmoid',
                                'steepness': 0.2    # Value is 'reshaped' over lifetime
                            }
                        }
                    },
                    {
                        'type': 'algae',
                        'value': 0.9,
                        'weighted': 'current_growth',
                        'flow_rate': {
                            'unit': 'kg',
                            'time': 'hour'
                        },
                        'criteria': {
                            'name': 'grown',
                            'limit': '=',
                            'value': True
                        }
                    },
#                    {
#                        'type': 'algae_waste',
#                        'value': 0.1,
#                        'weighted': 'current_growth',
#                        'flow_rate': {
#                            'unit': 'kg',
#                            'time': 'hour'
#                        },
#                        'criteria': {
#                            'name': 'grown',
#                            'limit': '=',
#                            'value': True
#                        }
#                    }
                ],
                'characteristics': [
                    {
                        'type': 'lifetime',
                        'value': 336,
                        'unit': 'hour'
                    },
                    {
                        'type': 'growth_criteria',
                        'value': 'out_biomass'
                    },
                    {
                        'type': 'capacity_biomass',
                        'value': 100,
                        'unit': 'kg'
                    },
                    {
                        'type': 'photoperiod',
                        'value': 16,
                        'unit': 'hour'
                    }
                ]
            }
        }
    }
}

agent_conn = [
    # Specify where to get/put currencies.
    # Note that 'greenhouse' and 'habitat' are special connections
    # which will be updated at initialization to match the selected
    # habitat and greenhouse agents (e.g. 'crew_habitat_small')
    {
        'from': 'greenhouse.co2',
        'to': 'algae.co2'
    },
    {
        'from': 'algae.o2',
        'to': 'greenhouse.o2'
    },
#    {
#        'from': 'nutrient_storage.nutrient_sol',
#        'to': 'algae.nutrient_sol'
#    },
    {
        'from': 'algae.biomass',
        'to': 'algae.biomass'
    },
    {
        'from': 'algae.algae',
        'to': 'food_storage.algae',
    },
#    {
#        'from': 'algae.algae_waste',
#        'to': 'nutrient_storage.algae_waste'
#    }
]

In [2]:
import json
with open('data_files/config_1hrad.json') as f:
    config = json.load(f)
    
# Add 10 square meters of algae to the simulation, More convient to measure in Liters.  
config['agents']['algae'] = {'amount': 10}
config['agents']['co2_storage'] = {'co2': 1000}
config['agents']['co2_makeup_valve'] = {'amount': 10}
# Start with 200 kg of inedible_biomass in storage
#config['agents']['nutrient_storage']['nutrient_sol'] = 200

from agent_model import AgentModel
model = AgentModel.from_config(config,
                               data_collection=True,
                               currency_desc=currency_desc,
                               agent_desc=agent_desc,
                               connections=agent_conn)
print(model)
model.step_to(n_steps=600)  # One past the agent's lifetime
data = model.get_data(debug=True)

FileNotFoundError: [Errno 2] No such file or directory: 'data_files/config_1hrad.json'

## Inspect a group
Plot all elements for one group with an agent.

Available groups vary by agent based on function, and may include: growth, storage, flows, deprive

In [3]:
import matplotlib.pyplot as plt

# A helper function to plot all elements in a group.
def plot_group(group, exclude=[], i=None, j=None):
    plt.figure(figsize=(12,6))
    length = len(next(iter(group.values())))
    i = i if i else 0
    j = j if j else length-1
    steps = [i for i in range(j-i)]
    for currency, values in group.items():
        if sum(values) == 0 or currency in exclude:
            continue
        plt.plot(steps, values[i:j], label=currency)
    plt.legend()
    plt.show()
    
# Uncomment lines below to see the results:

plot_group(data['algae']['flows'], exclude=['biomass','algae'])
plot_group(data['crew_habitat_small']['storage'], exclude=['h2o','n2','o2','biomass','algae'])
plot_currency(data,'co2')
plot_group(data['co2_storage']['storage'], exclude=[])
# plot_group(data['mushroom']['flows'], j=719)
# plot_group(data['mushroom']['growth'], exclude=['agent_step_num'])
# print(data.keys())
# print(data['mushroom'].keys())

ModuleNotFoundError: No module named 'matplotlib'

In [4]:
data['algae']['flows']['co2']

NameError: name 'data' is not defined

## Inspect a Currency
Plot all flows of a particular currency

In [6]:
import matplotlib.pyplot as plt

data = model.get_data(debug=True)
def plot_currency(data, currency, exclude=[], i=None, j=None):
    """Plot all agents who use a particular currency"""
    
    # Iterate through data and extract all records
    flows = {}
    length = None
    for agent_name, agent_data in data.items():
        if 'flow_records' not in agent_data:
            continue
        for currency_name, currency_data in agent_data['flow_records'].items():
            if currency_name != currency:
                continue
            flow_records = {}
            if not length:
                length = len(currency_data)
            for n, step in enumerate(currency_data):
                # Steps can include records for multiple currencies, 
                # or multiple records for the same currency
                at_least_one = False
                for record in step:
                    if agent_name not in flows:
                        flows[agent_name] = [] + [0] * n
                    if not at_least_one:
                        at_least_one = True
                        flows[agent_name].append(-record['amount'])
                    else:
                        flows[agent_name][-1] -= record['amount']
                if not at_least_one and agent_name in flows:
                    flows[agent_name].append(0)
    
    if not length:
        print("No flow records for", currency)
        return
    
    # Plot data
    plt.figure(figsize=(12,6))
    i = i if i else 0
    j = j if j else length
    steps = [x for x in range(j-i)]
    for agent_name, agent_data in flows.items():
        if agent_name in exclude:
            continue
        pad_zeros = len(steps) - len(agent_data)
        if pad_zeros > 0:
            agent_data += [0] * pad_zeros
        plt.plot(steps, agent_data[i:j], label=agent_name)
    plt.legend(loc='lower right')
    plt.show()
            
plot_currency(data, 'o2')
# plot_currency(data, 'potable', exclude=['multifiltration_purifier_post_treatment'])

ModuleNotFoundError: No module named 'matplotlib'